## CoastSeg
----

In [1]:
# Internal Imports
import warnings
# warnings.filterwarnings("ignore")

# External Imports
from IPython.display import display
from tkinter import Tk,filedialog
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from ipyleaflet import DrawControl, GeoJSON, LayersControl
import leafmap
import os

# Local Imports 
from CoastSeg import download_roi
from CoastSeg import bbox
from CoastSeg import make_overlapping_roi
from CoastSeg import zoo_model_module
from CoastSeg import file_functions

#suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
root = Tk()

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.



Version:  2.8.0
Eager mode:  True
Version:  2.8.0
Eager mode:  True
GPU name:  []
Num GPUs Available:  0


In [2]:
# Map Variables
# ---------------
center_point = ( 36, -121.5)
zoom = 13
ROI_SIZE = 0.008

map_settings={
"center_point": center_point,
"zoom":zoom,
 "draw_control":False,
 "measure_control":False, 
 "fullscreen_control":False, 
 "attribution_control":True,
 "Layout":Layout(width='100%', height='100px')
}

# CoastSat Download Variables
# ------------------------------
dates = ['2018-12-01', '2019-01-01']
sat_list = ['S2','L7','L8']

pre_process_settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
}

# Filenames to Store Data
# --------------------------
roi_filename = "official_roi.geojson"
csv_filename='overlap.csv'
selected_roi_file="selected_roi.geojson"
inputs_filename="inputs.json"

# Create the Map and Draw a Bounding Box

# Generate the ROIs in the Bounding Box
---
- Depending on how big the bounding box is it may take some time
- If you get a bouding box too large or too small error re-run the code to generate the bounding box

## Didn't get enough overlapping ROIs
1. Modify the percent overlap allowed parameter to be a smaller value than the default .65
-  Example where percent overlap allowed = ``.55``:
    ``overlap_btw_vectors_df=make_overlapping_roi.min_overlap_btw_vectors(roi_filename,csv_filename,.55)``
2. Comment out line: ``overlap_btw_vectors_df=make_overlapping_roi.min_overlap_btw_vectors(roi_filename,csv_filename,.65)``
to use the default method of generating overlap between ROIs. This may result in excessive overlap between rois and issues with too many rois clustering in one location.

# Click the ROIs you want to download
---
- If the bounding box is in the way of selecting the roi. Click the trash can icon on the left side menu, then click the bounding box, then click save. This will delete the bounding box and allow you to click an ROI.

In [3]:
class GeoJson_Map_Layers:
    shoreline_file=os.getcwd()+os.sep+"third_party_data"+os.sep+"stanford-xv279yj9196-geojson.json"
    def __init__(self, map_settings: dict):
        # data : geojson data of the rois generated
        self.data=None
        # selected_set : ids of the selected rois
        self.selected_set=set()
        # geojson_layer : layer with all rois
        self.geojson_layer=None
        # selected layer :  layer containing all selected rois
        self.selected_layer = None
        # shapes_list : Empty list to hold all the polygons drawn by the user
        self.shapes_list=[]
        # coastline_for_map : coastline vector geojson for map layer
        self.coastline_for_map=None
        # selected_ROI : Geojson for all the ROIs selected by the user
        self.selected_ROI=None
        
        self.m = leafmap.Map(draw_control=map_settings["draw_control"],
                        measure_control=map_settings["measure_control"],
                        fullscreen_control=map_settings["fullscreen_control"],
                        attribution_control=map_settings["attribution_control"],
                        center=map_settings["center_point"],
                        zoom=map_settings["zoom"],
                        layout=map_settings["Layout"])
        # Create drawing controls
        self.draw_control=self.create_DrawControl()
        self.draw_control.on_draw(self.handle_draw)
        self.m.add_control(self.draw_control)
        layer_control = LayersControl(position='topright')
        self.m.add_control(layer_control)
    
    
    def create_DrawControl(self):
        draw_control = DrawControl()
        draw_control.polyline = {}
        draw_control.circlemarker = {}
        # Custom styles for polygons and rectangles
        draw_control.polygon = {
            "shapeOptions": {
                "fillColor": "green",
                "color": "green",
                "fillOpacity": 0.2,
                "Opacity": 0.2
            },
            "drawError": {
                "color": "#dd253b",
                "message": "Ops!"
            },
            "allowIntersection": False,
            "transform":True
        }

        draw_control = DrawControl()
        draw_control.rectangle = {
            "shapeOptions": {
                "fillColor": "green",
                "color": "green",
                "fillOpacity": 0.1,
                "Opacity": 0.1
            },
            "drawError": {
                "color": "#dd253b",
                "message": "Ops!"
            },
            "allowIntersection": False,
            "transform":True
        }
        return  draw_control 
   
   
    def handle_draw(self,target, action, geo_json):
        self.action=action
        self.geo_json=geo_json
        self.target=target
        if self.draw_control.last_action == 'created'and self.draw_control.last_draw['geometry']['type']=='Polygon' :
            self.shapes_list.append( self.draw_control.last_draw['geometry'])
        if self.draw_control.last_action == 'deleted':
            self.shapes_list.pop()
    
    
    def set_data(self, roi_filename):
        # Read the geojson for all the ROIs generated
        self.data=download_roi.read_geojson_file(roi_filename)
        # Add style to each feature in the geojson
        for feature in self.data["features"]:
            feature["properties"]["style"] = {
                "color": "grey",
                "weight": 1,
                "fillColor": "grey",
                "fillOpacity": 0.2,
            }
    
    
    def generate_ROIS(self, roi_filename, csv_filename, progressbar):
        # Make sure your bounding box is within the allowed size
        bbox.validate_bbox_size(self.shapes_list)
        #dictionary containing geojson coastline
        roi_coastline=bbox.get_coastline(GeoJson_Map_Layers.shoreline_file,self.shapes_list)
        #coastline styled for the map
        self.coastline_for_map=self.get_coastline_layer(roi_coastline)
        self.m.add_layer(self.coastline_for_map)
        #Get the rois using the coastline  within bounding box
        geojson_polygons=make_overlapping_roi.get_ROIs(roi_coastline,roi_filename,csv_filename,progressbar)
        # Save the data from the ROI file to data
        self.set_data(roi_filename)
        # overlap_btw_vectors_df=make_overlapping_roi.min_overlap_btw_vectors(roi_filename,csv_filename,overlap_percent=.65)

    def save_roi_to_file(self, selected_roi_file):
        self.selected_ROI=download_roi.save_roi(roi_filename, selected_roi_file, self.selected_set)
  
  
    def get_coastline_layer(self,roi_coastline: dict):
        """Returns a GeoJSON object that can be added as layer to map """
        assert roi_coastline != {}, "ERROR.\n Empty geojson cannot be drawn onto  map"
        return GeoJSON(
            data=roi_coastline,
            name="Coastline",
            style={
                'color': 'yellow',
                'fill_color': 'yellow',
                'opacity': 1,
                'dashArray': '5',
                'fillOpacity': 0.5,
                'weight': 4},
            hover_style={
                'color': 'white',
                'dashArray': '4',
                'fillOpacity': 0.7},
        )  
    
    
    def get_geojson_layer(self):
        if self.geojson_layer is None:
             self.geojson_layer=GeoJSON(data=self.data, name="geojson data", hover_style={"fillColor": "red"})
        return self.geojson_layer
    
    
    def geojson_onclick_handler(self, event=None, id=None, properties=None, **args):
        if properties is None:
            return
        cid = properties["id"]
        self.selected_set.add(cid)
        if self.selected_layer is not None:
            self.m.remove_layer(self.selected_layer)
            
        self.selected_layer = GeoJSON(
            data=self.convert_selected_set_to_geojson(self.selected_set),
            name="Selected ROIs",
            hover_style={"fillColor": "blue"},
        )
        self.selected_layer.on_click(self.selected_onclick_handler)
        self.m.add_layer(self.selected_layer)
        
        
    def selected_onclick_handler(self,event=None, id=None, properties=None, **args):
        """This is the on click handler for a layer that is selected.
        This method removes the give layer's cid from the selected_set and removes the layer from
        select_layer."""
        if properties is None:
            return
        # Remove the current layers cid from selected set
        cid = properties["id"]
        self.selected_set.remove(cid)
        if self.selected_layer is not None:
            self.m.remove_layer(self.selected_layer)
        # Recreate the selected layers wihout the layer that was removed
        self.selected_layer = GeoJSON(
            data = self.convert_selected_set_to_geojson(self.selected_set),
            name="Selected ROIs",
            hover_style={"fillColor": "blue"},
        )
        # Recreate the onclick handler for the selected layers
        self.selected_layer.on_click(self.selected_onclick_handler)
        # Add selected layer to the map
        self.m.add_layer(self.selected_layer)
    
    
    def add_geojson_layer_to_map(self):
        geojson_layer =self.get_geojson_layer()
        geojson_layer.on_click(self.geojson_onclick_handler)
        self.m.add_layer(geojson_layer)
        
            
    def convert_selected_set_to_geojson(self,selected_set):
        geojson = {"type": "FeatureCollection", "features": []}
        # Select the geojson in the selected layer
        geojson["features"] = [
            feature
            for feature in self.data["features"]
            if feature["properties"]["id"] in selected_set
        ]
        # Modify geojson style for each polygon in the selected layer
        for feature in self.data["features"]:
            feature["properties"]["style"] = {
                "color": "blue",
                "weight": 2,
                "fillColor": "grey",
                "fillOpacity": 0.2,
            }
        return geojson


In [4]:
GML=GeoJson_Map_Layers(map_settings)
# Draw a bounding box where ROIs should be generated
# GML.m

In [5]:
gen_button = widgets.Button(description="Generate ROI")
save_button = widgets.Button(description="Save ROI")
# download_button = widgets.Button(description="Download ROI")
instructions = widgets.Output()
error_output = widgets.Output()
# Print the starting instructions to screen
with instructions:
            if GML.shapes_list == []:
                instructions.clear_output()
                print("Draw a bounding box on the coast first, then click Generate ROI.")
output_vbox=VBox([instructions,error_output],layout=Layout(padding='0px 0px 0px 80px'))
HBox([gen_button, save_button , output_vbox])

display(HBox([gen_button, save_button , output_vbox]))

progressbar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def on_gen_button_clicked(b):
    if GML.shapes_list == []:
        # instructions.clear_output()
        with instructions:
            instructions.clear_output()
            print("Draw a bounding box on the coast first, then click Generate ROI.")
    else:
        with instructions:
            instructions.clear_output()
            print("Generating ROIs please wait.") 
        # Generate ROIs along the coastline within the bounding box
        
        GML.generate_ROIS(roi_filename, csv_filename, progressbar)
        display(progressbar)
        # Add the Clickable ROIs to the map
        GML.add_geojson_layer_to_map()
        with instructions:
            instructions.clear_output()
            print("ROIs generated. Please Select at least one ROI and click Save ROI.")


def on_save_button_clicked(b):
    if len(GML.selected_set) == 0:
        with error_output:
            # error_output.clear_output()
            print("Must select at least 1 ROI first before you can save ROIs.")
    else:
        GML.save_roi_to_file(selected_roi_file)
        error_output.clear_output()
        with instructions:
            instructions.clear_output()
            print("Saving ROIs")
            instructions.clear_output()
            print("ROIs have been saved. Now click Download ROI to download the ROIs using CoastSat")

gen_button.on_click(on_gen_button_clicked)
save_button.on_click(on_save_button_clicked)
display(progressbar)
GML.m

FloatProgress(value=0.0, max=1.0)

Map(center=[36, -121.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [6]:
print(GML.selected_layer)
print(GML.selected_set)

GeoJSON(data={'type': 'FeatureCollection', 'features': [{"geometry": {"coordinates": [[[-121.691939, 36.136676], [-121.691939, 36.176676], [-121.651939, 36.176676], [-121.651939, 36.136676], [-121.691939, 36.136676]]], "type": "Polygon"}, "properties": {"id": 0, "style": {"color": "blue", "fillColor": "grey", "fillOpacity": 0.2, "weight": 2}}, "type": "Feature"}, {"geometry": {"coordinates": [[[-121.587696, 36.000209], [-121.587696, 36.040209], [-121.547696, 36.040209], [-121.547696, 36.000209], [-121.587696, 36.000209]]], "type": "Polygon"}, "properties": {"id": 4, "style": {"color": "blue", "fillColor": "grey", "fillOpacity": 0.2, "weight": 2}}, "type": "Feature"}, {"geometry": {"coordinates": [[[-121.498057, 35.921359], [-121.498057, 35.961359], [-121.458057, 35.961359], [-121.458057, 35.921359], [-121.498057, 35.921359]]], "type": "Polygon"}, "properties": {"id": 7, "style": {"color": "blue", "fillColor": "grey", "fillOpacity": 0.2, "weight": 2}}, "type": "Feature"}, {"geometry": {

## Get the ROIs selected by the user
- Make sure you click at least one roi before running the following code
1. Get the IDs of the ROI clicked by the user
2. Download the data associated with the ROIs using CoastSat
    - inputs_file is the json file where the input data for download imagery with CoastSat will be written to

In [7]:
download_roi.download_imagery(GML.selected_ROI,pre_process_settings,dates,sat_list,inputs_filename=inputs_filename)

2022-06-10 16:31:04.477399
Images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 15 images
  L7: 4 images
  L8: 4 images
  Total: 23 images
- In Landsat Tier 2:
  L7: 0 images
  L8: 2 images
  Total: 2 images
Images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 7 images
  L7: 4 images
  L8: 4 images
  Total: 15 images
- In Landsat Tier 2:
  L7: 0 images
  L8: 2 images
  Total: 2 images
Images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 13 images
  L7: 4 images
  L8: 4 images
  Total: 21 images
- In Landsat Tier 2:
  L7: 0 images
  L8: 2 images
  Total: 2 images
Images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 13 images
  L7: 4 images
  L8: 2 images
  Total: 19 images
- In Landsat Tier 2:
  L7: 0 images
  L8: 2 images
  Total: 2 images
Images available: 
 [{'polygon': [[[-121.691939, 36.136676


inputs:  {'polygon': [[[-121.691939, 36.136676], [-121.651939, 36.136676], [-121.651939, 36.176676], [-121.691939, 36.176676], [-121.691939, 36.136676]]], 'dates': ['2018-12-01', '2019-01-01'], 'sat_list': ['S2', 'L7', 'L8'], 'sitename': 'ID02022-06-10__16_hr_31_min', 'filepath': 'c:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\data'} 

Images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 15 images
  L7: 4 images
  L8: 4 images
  Total: 23 images
- In Landsat Tier 2:
  L7: 0 images
  L8: 2 images
  Total: 2 images

S2: 15 images
100%
L7: 4 images
100%
L8: 4 images
100%

metadata {'L7': {'filenames': ['2018-12-02-18-42-32_L7_ID02022-06-10__16_hr_31_min_pan.tif', '2018-12-11-18-36-07_L7_ID02022-06-10__16_hr_31_min_pan.tif', '2018-12-18-18-42-06_L7_ID02022-06-10__16_hr_31_min_pan.tif', '2018-12-27-18-35-39_L7_ID02022-06-10__16_hr_31_min_pan.tif'], 'acc_georef': [7.74399995803833, 10.11299991607666, 7.906000137329102, 4.9510

In [8]:
data_selection=widgets.Dropdown(
    options=[('Select Your Images', 1), ('Use Data Folder', 2)],
    value=1,
    description='Number:',
)

use_GPU_output = widgets.Output()
GPU_checkbox=widgets.Checkbox(
    value=False,
    description='Use GPU?',
    disabled=False,
    indent=False
)


In [10]:
#Rerun this to save the selections
display(data_selection, GPU_checkbox, use_GPU_output)

Dropdown(description='Number:', index=1, options=(('Select Your Images', 1), ('Use Data Folder', 2)), value=2)

Checkbox(value=False, description='Use GPU?', indent=False)

Output()

In [11]:
def get_jpgs_from_data():
    """Copies all the jpgs from the data folder in CoastSeg to a new folder, where the model
    will save the computed segmentations."""
    # Data folder location 
    src_path=os.getcwd()+os.sep+"data"
    if os.path.exists(src_path):
        file_functions.rename_jpgs(src_path)
        # Create a new folder to hold all the data
        location=os.getcwd()
        name="segmentation_data"
        new_folder=file_functions.mk_new_dir(name,location)
        glob_str = src_path + str(os.sep + "**" + os.sep) * 3 + "*jpg"
        file_functions.copy_files_to_dst(src_path,new_folder, glob_str)
        return new_folder
    else:
        print("ERROR: Cannot find the data directory in CoastSeg")
        raise Exception("ERROR: Cannot find the data directory in CoastSeg")


In [12]:
if data_selection.value == 1:
    # Prompt the user to select a directory of images                              
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    root.filename =  filedialog.askdirectory(initialdir = "/samples",title = "Select directory of images (or npzs) to segment")
    # Save the filename as an attribute of the button
    if root.filename:
        sample_direc= root.filename
        print(f"{sample_direc} will be segmented")
    else:
        raise Exception("You must select a valid directory first!")
elif data_selection.value ==2:
    # Use the data folder as the input for segmentation
    print("Loading in the jpgs from the data directory")
    # Copy the jpgs from data to a new folder called segmentation_data_[datetime]
    sample_direc = get_jpgs_from_data()
    print(f"Contents of the data directory saved in {sample_direc}")

Loading in the jpgs from the data directory
Renamed files in c:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\data 
Renamed files in c:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\data 
Renamed files in c:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\data 
Renamed files in c:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\data 
2022-06-10 16:39:29.004739
Copied files that matched c:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\data\**\\**\\**\*jpg  to c:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\segmentation_data_2022-06-10__16_hr_39_min
Contents of the data directory saved in c:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\segmentation_data_2022-06-10__16_hr_39_min


In [13]:
if GPU_checkbox.value == False:
    print("Not using the GPU")
    ## to use the CPU (not recommended):
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
else:
    print("Using the GPU")
    ##use the first available GPU
    os.environ['CUDA_VISIBLE_DEVICES'] = '0' #'1'

Not using the GPU


In [14]:
dataset='RGB'
dataset_id ='landsat_6229071'
model_choice='ENSEMBLE'
zoo_model=zoo_model_module.Zoo_Model()
# # First download the specified model
zoo_model.download_model(dataset,dataset_id)
# # Get weights as list
Ww=zoo_model.get_weights_list(model_choice)
# Load the model from the config files
model, model_list, config_files, model_types=zoo_model.get_model(Ww)
metadatadict=zoo_model.get_metadatadict(Ww,config_files, model_types)
# # Compute the segmentation
zoo_model.compute_segmentation(sample_direc, model_list, metadatadict)


100%|██████████| 57/57 [05:31<00:00,  5.81s/it]
